In [15]:
import pandas as pd
import nltk
import re

from typing import List

In [16]:
# DUPLICATES 

In [17]:
MEASURES = ['cup', 'tsp', 'tbsp', 'c' , 'fl oz', 'pt', 'qt', 'gal', 'g', 'kg', 'mg', 'oz', 'lb', 'slice', 'piece', 'pinch', 'dash', 'whole', 'dozen', 'count', 'pkg', 'can', 'jar', 'carton', 'stick', 'drop', 'cm', 'm', 'ml', 'l', 'x', 'pint', 'cubes', 'cube', 'pack']
# STOP_WORDS = ['a', 'an', 'the', 'in', 'on', 'at', 'for', 'of', 'with', 'to', 'from', 'by', 'as', 'is', 'are', 'was', 'were', 'has', 'have', 'had', 'need']
STOP_WORDS = ['new', "%", "¼", "½"]

DATA_CSV_PATH = r'/Users/arina/study/ds/project/food_recommendation/input/bcc_goodfood/data_goodfood_bbc_short.csv' 
DATA_PARSED_CSV_PATH = r'/Users/arina/study/ds/project/food_recommendation/input/bcc_goodfood/data_goodfood_bbc_short_parsed.csv' 

In [18]:
df = pd.read_csv(DATA_CSV_PATH, sep='\t')

In [19]:
recipe_df = df.copy()
recipe_df

,type,name,ingredients,difficulty,health_banners
0,afternoon-tea,Lemon drizzle cake,"['225g unsalted butter, softened', '225g cast...",Easy,['Freezable']
1,afternoon-tea,Best ever chocolate brownies recipe,"['185g unsalted butter', '185g best dark choco...",More effort,[]
2,afternoon-tea,Classic Victoria sandwich recipe,"['200g caster sugar', '200g softened butter', ...",Easy,['Freezable (Cake base freezes well for 3 mont...
3,afternoon-tea,Easy chocolate fudge cake,"['150ml sunflower oil, plus extra for the tin'...",Easy,['Vegetarian']
4,afternoon-tea,Classic scones with jam & clotted cream,"['350g self-raising flour, plus more for dusti...",Easy,"['Easily doubled', 'Easily halved', 'Freezable']"
...,...,...,...,...,...
623,vegetable,Parsnip & celeriac bake,"['600g parsnip, peeled and cubed', '1kg celeri...",Easy,"['Freezable (before step 2 )', 'Healthy', 'Veg..."
624,vegetable,Winter greens puff pie,"['1 tbsp olive oil', '1 onion, chopped', '2 ga...",Easy,['Vegetarian']
625,vegetable,Asian greens,"['1 tbsp vegetable oil', '2 tbsp oyster sauce'...",Easy,[]
626,vegetable,Giant butter bean stew,['4 x 235g cans butter beans or 500g dried but...,Easy,"['Freezable', 'Gluten-free', 'Healthy', 'Veget..."


In [20]:
recipe_df.shape

(628, 5)

In [21]:
recipe_df.sort_values("name", inplace=True)
recipe_df.drop_duplicates(subset="name",
                    keep=False, inplace=True)
recipe_df

,type,name,ingredients,difficulty,health_banners
345,lunch,10-minute couscous salad,"['100g couscous', '200ml hot low salt vegetabl...",Easy,['Vegetarian']
203,condiment,Angela's 'flexible' jam recipe,"['900g fruit (blackberries, plums, raspberries...",Easy,[]
191,condiment,Apple & cranberry chutney,"['1kg cooking apples, peeled and chopped into ...",Easy,['Vegetarian']
625,vegetable,Asian greens,"['1 tbsp vegetable oil', '2 tbsp oyster sauce'...",Easy,[]
403,pasta,Asparagus cream pasta,"['1 bunch asparagus', '142ml tub double cream...",Easy,['Vegetarian']
...,...,...,...,...,...
281,drink,White lady,"['50ml gin', '25ml triple sec', '25ml lemon ju...",Easy,"['Gluten-free', 'Vegetarian']"
172,cocktails,White rabbit cocktail,"['80ml gin', '50ml amaretto', '50ml double cre...",Easy,"['Gluten-free', 'Vegetarian']"
201,condiment,White sauce,"['500ml whole milk', '1 onion, halved', '1 bay...",Easy,"['Freezable', 'Vegetarian']"
424,side-dish,White sourdough,"['250g strong white bread flour, preferably or...",A challenge,"['Vegan', 'Vegetarian']"


In [22]:
recipe_df.shape

(263, 5)

In [23]:
recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263 entries, 345 to 624
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   type            263 non-null    object
 1   name            263 non-null    object
 2   ingredients     263 non-null    object
 3   difficulty      263 non-null    object
 4   health_banners  263 non-null    object
dtypes: object(5)
memory usage: 12.3+ KB


In [24]:
recipe_df.index = range(len(recipe_df))
recipe_df.head() 

,type,name,ingredients,difficulty,health_banners
0,lunch,10-minute couscous salad,"['100g couscous', '200ml hot low salt vegetabl...",Easy,['Vegetarian']
1,condiment,Angela's 'flexible' jam recipe,"['900g fruit (blackberries, plums, raspberries...",Easy,[]
2,condiment,Apple & cranberry chutney,"['1kg cooking apples, peeled and chopped into ...",Easy,['Vegetarian']
3,vegetable,Asian greens,"['1 tbsp vegetable oil', '2 tbsp oyster sauce'...",Easy,[]
4,pasta,Asparagus cream pasta,"['1 bunch asparagus', '142ml tub double cream...",Easy,['Vegetarian']


In [25]:
recipe_df = recipe_df.loc[:,['type','name','ingredients']]

In [26]:
# https://archive.nytimes.com/open.blogs.nytimes.com/2015/04/09/extracting-structured-data-from-recipes-using-conditional-random-fields/

In [27]:
recipe_df[['ingredients']]

,ingredients
0,"['100g couscous', '200ml hot low salt vegetabl..."
1,"['900g fruit (blackberries, plums, raspberries..."
2,"['1kg cooking apples, peeled and chopped into ..."
3,"['1 tbsp vegetable oil', '2 tbsp oyster sauce'..."
4,"['1 bunch asparagus', '142ml tub double cream..."
...,...
258,"['50ml gin', '25ml triple sec', '25ml lemon ju..."
259,"['80ml gin', '50ml amaretto', '50ml double cre..."
260,"['500ml whole milk', '1 onion, halved', '1 bay..."
261,"['250g strong white bread flour, preferably or..."


In [28]:
recipe_df['ingredients'][0]

"['100g couscous', '200ml hot low salt vegetable stock (from a cube is fine)', '2 spring onions', '1 red pepper', '½ cucumber', '50g feta cheese, cubed ', '2tbsp pesto', '2tbsp pine nuts']"

In [29]:
# удаляю цифры + текст в скобках + u2028
recipe_df["ingredients"] = recipe_df["ingredients"].str.replace(r'(\d|\½|¼)(\-?\.?)\s?|\((.*?)\)|\\u', "")
recipe_df["ingredients"]

/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_20382/1583251949.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  recipe_df["ingredients"] = recipe_df["ingredients"].str.replace(r'(\d|\½|¼)(\-?\.?)\s?|\((.*?)\)|\\u', "")


0      ['g couscous', 'ml hot low salt vegetable stoc...
1      ['g fruit , prepared weight', 'g golden granul...
2      ['kg cooking apples, peeled and chopped into s...
3      ['tbsp vegetable oil', 'tbsp oyster sauce', 'g...
4      [' bunch asparagus', 'ml tub double cream', 'g...
                             ...                        
258    ['ml gin', 'ml triple sec', 'ml lemon juice, p...
259    ['ml gin', 'ml amaretto', 'ml double cream', '...
260    ['ml whole milk', 'onion, halved', 'bay leaf',...
261    ['g strong white bread flour, preferably organ...
262    ['tbsp olive oil', 'onion, chopped', 'garlic c...
Name: ingredients, Length: 263, dtype: object

In [30]:
# measures
pattern = r'\b(?:{})\s?\b'.format('|'.join(MEASURES))
recipe_df["ingredients"] = recipe_df["ingredients"].str.replace(pattern, "")
recipe_df["ingredients"]

/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_20382/1986964682.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  recipe_df["ingredients"] = recipe_df["ingredients"].str.replace(pattern, "")


0      ['couscous', 'hot low salt vegetable stock ', ...
1      ['fruit , prepared weight', 'golden granulated...
2      ['cooking apples, peeled and chopped into smal...
3      ['vegetable oil', 'oyster sauce', 'mixed Asian...
4      [' bunch asparagus', 'tub double cream', 'garl...
                             ...                        
258    ['gin', 'triple sec', 'lemon juice, plus lemon...
259    ['gin', 'amaretto', 'double cream', 'lemon cur...
260    ['milk', 'onion, halved', 'bay leaf', 'cloves'...
261    ['strong white bread flour, preferably organic...
262    ['olive oil', 'onion, chopped', 'garlic cloves...
Name: ingredients, Length: 263, dtype: object

In [31]:
nltk.download('stopwords')
stop_words_built_in = nltk.corpus.stopwords.words('english')
stop_words = set(stop_words_built_in + STOP_WORDS)
stop_words


[nltk_data] Downloading package stopwords to /Users/arina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'%',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'new',
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over

In [32]:
# stop_words

pattern = r'\b(?:{})\s?\b'.format('|'.join(stop_words))

# pattern = r'\b(?:{})\s?\b'.format('|'.join(stop_words_built_in))
recipe_df["ingredients"] = recipe_df["ingredients"].str.replace(pattern, "")
recipe_df["ingredients"]


/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_20382/219454158.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  recipe_df["ingredients"] = recipe_df["ingredients"].str.replace(pattern, "")


0      ['couscous', 'hot low salt vegetable stock ', ...
1      ['fruit , prepared weight', 'golden granulated...
2      ['cooking apples, peeled chopped small chunks'...
3      ['vegetable oil', 'oyster sauce', 'mixed Asian...
4      [' bunch asparagus', 'tub double cream', 'garl...
                             ...                        
258    ['gin', 'triple sec', 'lemon juice, plus lemon...
259    ['gin', 'amaretto', 'double cream', 'lemon cur...
260    ['milk', 'onion, halved', 'bay leaf', 'cloves'...
261    ['strong white bread flour, preferably organic...
262    ['olive oil', 'onion, chopped', 'garlic cloves...
Name: ingredients, Length: 263, dtype: object

In [33]:
recipe_df["ingredients"] = recipe_df["ingredients"].str.lower()
recipe_df["ingredients"]

0      ['couscous', 'hot low salt vegetable stock ', ...
1      ['fruit , prepared weight', 'golden granulated...
2      ['cooking apples, peeled chopped small chunks'...
3      ['vegetable oil', 'oyster sauce', 'mixed asian...
4      [' bunch asparagus', 'tub double cream', 'garl...
                             ...                        
258    ['gin', 'triple sec', 'lemon juice, plus lemon...
259    ['gin', 'amaretto', 'double cream', 'lemon cur...
260    ['milk', 'onion, halved', 'bay leaf', 'cloves'...
261    ['strong white bread flour, preferably organic...
262    ['olive oil', 'onion, chopped', 'garlic cloves...
Name: ingredients, Length: 263, dtype: object

In [34]:
recipe_df

,type,name,ingredients
0,lunch,10-minute couscous salad,"['couscous', 'hot low salt vegetable stock ', ..."
1,condiment,Angela's 'flexible' jam recipe,"['fruit , prepared weight', 'golden granulated..."
2,condiment,Apple & cranberry chutney,"['cooking apples, peeled chopped small chunks'..."
3,vegetable,Asian greens,"['vegetable oil', 'oyster sauce', 'mixed asian..."
4,pasta,Asparagus cream pasta,"[' bunch asparagus', 'tub double cream', 'garl..."
...,...,...,...
258,drink,White lady,"['gin', 'triple sec', 'lemon juice, plus lemon..."
259,cocktails,White rabbit cocktail,"['gin', 'amaretto', 'double cream', 'lemon cur..."
260,condiment,White sauce,"['milk', 'onion, halved', 'bay leaf', 'cloves'..."
261,side-dish,White sourdough,"['strong white bread flour, preferably organic..."


In [35]:
recipe_df['ingredients'] = recipe_df['ingredients'].apply(eval).tolist()
recipe_df['ingredients']

0      [couscous, hot low salt vegetable stock , spri...
1      [fruit , prepared weight, golden granulated su...
2      [cooking apples, peeled chopped small chunks, ...
3      [vegetable oil, oyster sauce, mixed asian gree...
4      [ bunch asparagus, tub double cream, garlic cl...
                             ...                        
258    [gin, triple sec, lemon juice, plus lemon zest...
259    [gin, amaretto, double cream, lemon curd,   va...
260    [milk, onion, halved, bay leaf, cloves, butter...
261    [strong white bread flour, preferably organic ...
262    [olive oil, onion, chopped, garlic cloves, cru...
Name: ingredients, Length: 263, dtype: object

In [36]:
recipe_df['ingredients'][0]

['couscous',
 'hot low salt vegetable stock ',
 'spring onions',
 'red pepper',
 'cucumber',
 'feta cheese, cubed ',
 'pesto',
 'pine nuts']

In [37]:
# # пунктуация!
# (r'[-\s.,;!?]+')

In [38]:
# from nltk.tokenize import RegexpTokenizer

In [39]:
# tokenizer = RegexpTokenizer(r'[-\s.,;!?]+')
# tokenizer.tokenize(sentence)

In [40]:
# recipe_df["ingredients"] = recipe_df["ingredients"].str.replace(r'[-.,;!?]+', "")
# recipe_df["ingredients"]

In [41]:
# import nltk

In [42]:
# from nltk.tokenize import TreebankWordTokenizer
# tokenizer = TreebankWordTokenizer()

Нормализация особенно полезна для поисковых систем. При поиске нормали- зация увеличивает количество найденных совпадений для конкретного запроса. 

In [43]:
# import nltk
# nltk.download('wordnet')
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer

In [44]:
# # SnowballStemmer - более "сильный"

In [45]:
lemmatizer = WordNetLemmatizer()

In [46]:
stemmer = SnowballStemmer("english")

In [47]:
def parse_list_ingredients(list_ingredients: List) -> List:
    result = []
    for ingredients in list_ingredients:
        words = re.split(r'\W+|\s+', ingredients)
        # words = [lemmatizer.lemmatize(word) for word in words]
        words = [stemmer.stem(word) for word in words]
        if words:
            result.append(' '.join(words)) 
    return result

In [48]:
recipe_df['ingredients_parsed'] = recipe_df['ingredients'].apply(lambda x: parse_list_ingredients(x))
recipe_df['ingredients_parsed']

0      [couscous, hot low salt veget stock , spring o...
1      [fruit prepar weight, golden granul sugar, kno...
2      [cook appl peel chop small chunk, eat appl pee...
3      [veget oil, oyster sauc, mix asian green pak c...
4      [ bunch asparagus, tub doubl cream, garlic clo...
                             ...                        
258    [gin, tripl sec, lemon juic plus lemon zest tw...
259    [gin, amaretto, doubl cream, lemon curd,  vani...
260    [milk, onion halv, bay leaf, clove, butter, pl...
261    [strong white bread flour prefer organ stonegr...
262    [oliv oil, onion chop, garlic clove crush, gen...
Name: ingredients_parsed, Length: 263, dtype: object

In [49]:
recipe_df

,type,name,ingredients,ingredients_parsed
0,lunch,10-minute couscous salad,"[couscous, hot low salt vegetable stock , spri...","[couscous, hot low salt veget stock , spring o..."
1,condiment,Angela's 'flexible' jam recipe,"[fruit , prepared weight, golden granulated su...","[fruit prepar weight, golden granul sugar, kno..."
2,condiment,Apple & cranberry chutney,"[cooking apples, peeled chopped small chunks, ...","[cook appl peel chop small chunk, eat appl pee..."
3,vegetable,Asian greens,"[vegetable oil, oyster sauce, mixed asian gree...","[veget oil, oyster sauc, mix asian green pak c..."
4,pasta,Asparagus cream pasta,"[ bunch asparagus, tub double cream, garlic cl...","[ bunch asparagus, tub doubl cream, garlic clo..."
...,...,...,...,...
258,drink,White lady,"[gin, triple sec, lemon juice, plus lemon zest...","[gin, tripl sec, lemon juic plus lemon zest tw..."
259,cocktails,White rabbit cocktail,"[gin, amaretto, double cream, lemon curd, va...","[gin, amaretto, doubl cream, lemon curd, vani..."
260,condiment,White sauce,"[milk, onion, halved, bay leaf, cloves, butter...","[milk, onion halv, bay leaf, clove, butter, pl..."
261,side-dish,White sourdough,"[strong white bread flour, preferably organic ...",[strong white bread flour prefer organ stonegr...


In [50]:
recipe_df.to_csv(DATA_PARSED_CSV_PATH, sep="\t", index=False)